In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import gc
from kaggle_datasets import KaggleDatasets
from tokenizers import ByteLevelBPETokenizer
import os
from tqdm import tqdm
import time

In [2]:
img_directory = '../input/flickr8k/Images'
images_path = []

for file in os.listdir(img_directory):
    if file == 'flickr30k_images' or file == 'results.csv':
        continue
    images_path.append(os.path.join(img_directory, file))
images_path[0]

'../input/flickr8k/Images/3226254560_2f8ac147ea.jpg'

In [3]:
images_path[0].split('/')[4]

'3226254560_2f8ac147ea.jpg'

In [4]:
len(images_path)

8091

In [5]:
# some image parameters
img_height = 256
img_width = 256
n_channels = 3

In [6]:
from tensorflow.keras.applications import efficientnet
base_model = efficientnet.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(img_height, img_width, n_channels))
base_model.trainable = False
base_model_out = base_model.layers[-1].output
base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
cnn_model = keras.models.Model(base_model.input, base_model_out)

2022-08-28 18:41:59.423739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-28 18:41:59.547326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-28 18:41:59.548163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-28 18:41:59.549937: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

16719872/16705208 [==============================] - 0s 0us/step


In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def preprocess_image(image_directory):
    """Preprocesses the images into format expected by the InceptionV3 model
    
    expects image_directory to be a list of all the image_path to use
    """
    feature_dict = {}
    for img in tqdm(image_directory):
        # get the image id
        img_id = img.split('/')[4]
        
        # load the image and convert to array
        img = load_img(img, target_size=(img_height, img_width))
        img = img_to_array(img)
        
        # preprocess
        img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
        #img = tf.keras.applications.vgg16.preprocess_input(img)
        
        # extract and store in dictionary
        feature = cnn_model(img)
        #batch_features = tf.reshape(feature, (-1, feature.shape[3]))
        feature_dict[img_id] = feature.numpy()
        
        del img, feature
        gc.collect()
        
    return feature_dict

In [8]:
feature_dict = preprocess_image(images_path)

100%|██████████| 8091/8091 [34:30<00:00,  3.91it/s]


In [9]:
gc.collect()

23

In [10]:
# save for future references
import pickle

with open('feature_dict.pickle', 'wb') as handle:
    pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)